In [4]:
import pandas as pd
import os
import requests
from pprint import pprint
from config import g_key
import gmaps

In [3]:
path = os.path.join('..','WeatherPy','cities_df.csv')
df=pd.read_csv(path)
df.head()

,Cities,Latitude,Longitude,Cloudiness,Date,Humidity,Max_Temp,Wind Speed
0,esmeraldas,-19.76,-44.31,0,1600027988,32,82.40,10.29
1,torbay,47.67,-52.73,75,1600027988,44,62.01,9.17
2,areosa,41.72,-8.86,100,1600027988,78,75.00,8.05
3,setermoen,68.86,18.35,75,1600027989,93,46.40,4.70
4,port lincoln,-34.73,135.87,3,1600027989,89,52.61,5.08


In [6]:
gmaps.configure(api_key=g_key)

In [19]:
locations=df[['Latitude','Longitude']]
humidity=df['Humidity'].astype(float)
maxx=df['Humidity'].max()
maxx=maxx.astype(float)

In [20]:
fig=gmaps.figure()

In [21]:
heat_layer=gmaps.heatmap_layer(locations,weights=humidity,dissipating=False,max_intensity=maxx,point_radius=0.9)
fig.add_layer(heat_layer)

In [15]:
fig

Figure(layout=FigureLayout(height='420px'))

In [35]:
#Finding ideal conditions
df_hotels=df[(df['Max_Temp']>70) & (df['Max_Temp']<80) & (df.Cloudiness == 0) & (df['Wind Speed']<10)]

In [37]:
df_hotels.shape

(12, 8)

In [39]:
df_hotels.reset_index(inplace=True,drop=True)